In [1]:
import pandas as pd
import argparse
import logging
import json
import os
import pathlib
import fiona
import numpy as np
import geopandas as gpd
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from geopandas.tools import geocode
from shapely.geometry import Point
import requests
from zipfile import ZipFile
from tqdm import trange, tqdm

#import geojson

In [2]:
def filter_bldg_type_district(bldgs, gklas, gstat):
    bldg_res = bldgs.loc[(bldgs["gstat"]== 1004)]
    return bldg_res

In [30]:
# Define constants
EPSG_CODE = "EPSG:2056"
# BUFFER_DISTANCE = 5
GKLAS = [1110,1121,1122,1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1241, 1242, 1271, 1274, 1275, 1276, 1277, 1278, 1231, 1252, 1273]
GKLAS_HTG = [1110, 1121, 1122, 1130, 1211, 1212, 1220, 1230, 1251, 1261, 1262, 1263, 1264, 1265, 1272, 1231]
GKLAS_NOHTG = [1241, 1242, 1252, 1271, 1273, 1274, 1275, 1276, 1277, 1278]
GBAUP = [8011,8012,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022,8023]
GSTAT = 1004 # existing

REA_CODES_DESIRED_CC = ['EGID', 'GKODE', 'GKODN',
                           'GSTAT', 'GKAT', 'GKLAS', 'GBAUJ', 'GBAUP',  'GABBJ', 'GAREA','GASTW', 
                           'GWAERZH1', 'GENH1', 'GWAERDATH1', 'GWAERZH2', 'GENH2', 'GWAERDATH2', 
                           'GWAERZW1', 'GENW1', 'GWAERDATW1','GWAERZW2', 'GENW2', 'GWAERDATW2', 
                            'GGDENR', 'GGDENAME', 'GDEKT'] #'GRENJ','Create_Date','Update_Date',

REA_CODES_DESIRED_LC = ['egid', 'strname_deinr', 'ggdename', 'ggdenr',
       'gexpdat', 'gdekt', 'egrid','gebnr', 'gkode', 'gkodn', 'gksce', 'gstat', 'gkat', 'gklas',
       'gbauj', 'gbaup', 'gabbj', 'garea', 'gastw', 'gazzi', 'gebf', 'gwaerzh1',
       'genh1', 'gwaersceh1', 'gwaerdath1', 'gwaerzh2', 'genh2', 'gwaersceh2',
       'gwaerdath2', 'gwaerzw1', 'genw1', 'gwaerscew1', 'gwaerdatw1',
       'gwaerzw2', 'genw2', 'gwaerscew2', 'gwaerdatw2']

In [6]:
pathlib.Path()
NOTEBOOK_PATH = pathlib.Path().resolve()
p = NOTEBOOK_PATH.parent
DATA_DIRECTORY = p / "data"
MAP_DIRECTORY = DATA_DIRECTORY /'maps'
BLDG_DIRECTORY = DATA_DIRECTORY /'Buildings_rea'

#SELECT CANTON
#CANTON = "Genève"
CANTON = "Ticino"

# Read the shp file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
SWISSTOPO_DISTRICT_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_BEZIRKSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html
SWISSTOPO_CANTON_FILE = "swissboundaries3d_2023-01_2056_5728.shp/swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp" #shapefile downloaded from https://www.swisstopo.admin.ch/fr/geodata/landscape/boundaries3d.html. NOTE: The actual shapefile (.shp) is useless without the companion files: .dbf, .shx, .prj etc..
### SWISSTOPO_CANTON_FILE = figureout how to get the municipal boundaries https://api3.geo.admin.ch/rest/services/api/MapServer/ch.swisstopo.swissboundaries3d-gemeinde-flaeche.fill

CANTON_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_CANTON_FILE
DISTRICT_FILE_PATH = MAP_DIRECTORY / SWISSTOPO_DISTRICT_FILE

CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON

# Read the gpkg file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
PLANT_DIRECTORY = DATA_DIRECTORY /'ElectricityProduction_EPSG2056'

In [7]:
canton_raw_limits = gpd.read_file(CANTON_FILE_PATH)
canton_limits = canton_raw_limits.to_crs(epsg=2056)
canton_selection = canton_limits[canton_limits.NAME == CANTON]
canton_selection = canton_selection[["NAME", "geometry"]]
canton_selection= canton_selection.reset_index()
canton_selection.at[0,"geometry"]
print(canton_selection)

   index    NAME                                           geometry
0      4  Ticino  POLYGON Z ((2727358.511 1119219.094 1351.527, ...


In [8]:
# Read the shp file and decode the Geopandas dataframe using the Swiss coordinates (epsg code: 2056)
district_raw_limits = gpd.read_file(MAP_DIRECTORY/SWISSTOPO_DISTRICT_FILE)
district_limits = district_raw_limits.to_crs(epsg=2056)

In [9]:
#TI_districts = ["Bellinzona", "Lugano", "Mendrisio", "Locarno", "Vallemaggia", "Leventina", "Blenio", "Riviera"]
#District = district_limits[district_limits['NAME'].isin(TI_districts)]
Districts = district_limits[district_limits['KANTONSNUM'] == 21]
Districts = Districts[["NAME", "geometry"]].reset_index()
Districts

,index,NAME,geometry
0,13,Locarno,"POLYGON Z ((2714379.965 1113365.731 199.628, 2..."
1,16,Vallemaggia,"POLYGON Z ((2697839.712 1117891.512 1382.656, ..."
2,23,Leventina,"POLYGON Z ((2690167.460 1159217.479 2999.313, ..."
3,31,Blenio,"POLYGON Z ((2722946.495 1150435.679 3390.416, ..."
4,32,Lugano,"POLYGON Z ((2705224.608 1096540.957 691.169, 2..."
5,54,Bellinzona,"POLYGON Z ((2727358.511 1119219.094 1351.527, ..."
6,76,Riviera,"POLYGON Z ((2716627.405 1134972.637 286.404, 2..."
7,102,Mendrisio,"POLYGON Z ((2716548.012 1080243.765 440.335, 2..."


# 1. Comparing th dowloaded REA csv and geojson file

# 1.1 Getting the data from the geojson

In [10]:
CANTON_CSV_DIRECTORY = BLDG_DIRECTORY / CANTON
file = CANTON_CSV_DIRECTORY / "buildings.geojson"
TI = gpd.read_file(file)

## 1.2 Getting data from downloaded cantonal file ()

In [12]:
file_csv = CANTON_CSV_DIRECTORY / "REA_TI.csv"
REA_TI = pd.read_csv(file_csv)
REA_TI = REA_TI[REA_CODES_DESIRED_CC]
REA_TI.head()

,EGID,GKODE,GKODN,GSTAT,GKAT,GKLAS,GBAUJ,GBAUP,GABBJ,GAREA,...,GWAERDATH2,GWAERZW1,GENW1,GWAERDATW1,GWAERZW2,GENW2,GWAERDATW2,GGDENR,GGDENAME,GDEKT
0,11101225,2723565.29,1118776.54,1004.0,1060.0,NaN,NaN,NaN,NaN,806.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5001,Arbedo-Castione,TI
1,11101230,2723829.13,1119297.01,1004.0,1060.0,NaN,2023.0,8023.0,NaN,378.0,...,NaN,7630.0,7530.0,11/05/2015,NaN,NaN,NaN,5001,Arbedo-Castione,TI
2,11101231,2723945.82,1119821.41,1004.0,1060.0,NaN,NaN,8017.0,NaN,798.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5001,Arbedo-Castione,TI
3,11101233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5001,Arbedo-Castione,TI
4,11101236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5001,Arbedo-Castione,TI


In [13]:
REA_TI_EXIST = REA_TI.loc[REA_TI["GSTAT"] == 1004]
len(REA_TI_EXIST)/len(REA_TI)

0.9670960399795484

In [14]:
def get_demand_coefficients():
    
    fileDir = DATA_DIRECTORY / "support_data" 
    # reading consumption data for SH and DHW per type of building and construction time
    data_sh = pd.read_csv(fileDir/"demand_SH.csv" , header=0, index_col=0)
    data_dhw = pd.read_csv(fileDir/"demand_DHW.csv", header=0, index_col=0)
    fehh = pd.read_csv(fileDir/"fehh.csv", header=0, index_col=0)
    
    return data_sh, data_dhw, fehh

In [15]:
def get_dataframe(GGDENR):
    buildings = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"].isin(GGDENR)]
    return buildings

In [181]:
def post_processing(buildings):
    
    age_group = buildings[["GKAT", "GKLAS", "GBAUP"]]
    age_pivot = pd.pivot_table(age_group, values='GBAUP', index=['GKAT'], columns=['GKLAS'], aggfunc=np.mean)
    age_pivot = age_pivot.round(0)
    col = list(age_pivot.columns)
    for code in GKLAS:
        if code in col:
            pass
        else:
            age_pivot[code] = 8016.0

    size_group = buildings[["GKAT", "GKLAS", "GBAUP"]]
    size_pivot = pd.pivot_table(size_group, values='GBAUP', index=['GKAT'], columns=['GKLAS'], aggfunc=np.mean)
    size_pivot = size_pivot.round(0)  
  
    buildings["GKLAS_fill"] = buildings["GKLAS"]
    buildings["GBAUP_fill"] = buildings["GBAUP"]
    buildings["GAREA_fill"] = buildings["GAREA"]
    buildings["GASTW_fill"] = buildings["GASTW"]

    for g in range(len(buildings)):
        if (np.isnan(buildings.at[g,"GKLAS"])):
            if (buildings.at[g,"GKAT"] == 1020.0):
                buildings.loc[g,"GKLAS_fill"] = 1110.0
            elif (buildings.at[g,"GKAT"] == 1030.0):
                buildings.loc[g,"GKLAS_fill"] = 1122.0 # 63% of known 1040
            elif (buildings.at[g,"GKAT"] == 1040.0):
                buildings.loc[g,"GKLAS_fill"] = 1211.0  # 37% of known 1040
            elif (buildings.at[g,"GKAT"] == 1060.0):
                buildings.loc[g,"GKLAS_fill"] =(1271.0) # 31% of known 1060
                
                 
        if np.isnan(buildings.at[g,"GBAUP"]):         
            buildings.at[g,"GBAUP_fill"] = age_pivot.loc[(buildings.at[g,"GKAT"],buildings.at[g,"GKLAS_fill"])]
        
        if np.isnan(buildings.at[g,"GAREA"]):         
            buildings.at[g,"GAREA_fill"] = size_pivot.loc[(buildings.at[g,"GKAT"],buildings.at[g,"GKLAS_fill"])]
        
    b["GAREA_fill"] = b["GAREA_fill"].astype(np.int64)
    b["GKLAS_fill"] = b["GKLAS_fill"].astype(np.int64)
    b["GBAUP_fill"] = b["GBAUP_fill"].astype(np.int64)
    b["GASTW_fill"] = b["GASTW_fill"].fillna(value = 1)

    for g in range(len(b)):
        buildings.at[g,'k_SH'] = data_sh.loc[(buildings.at[g,"GBAUP_fill"],str(buildings.at[g,"GKLAS_fill"]))]
        buildings.at[g,'k_DHW'] = data_dhw.loc[(buildings.at[g,"GBAUP_fill"],str(buildings.at[g,"GKLAS_fill"]))]
        buildings.at[g,'k_fehh'] = fehh.loc[(buildings.at[g,"GBAUP_fill"],str(buildings.at[g,"GKLAS_fill"]))]

    # calculating building attributes and demand
    buildings['ERA'] = buildings['GAREA_fill']*buildings['GASTW_fill']
    buildings['SH'] = buildings['k_SH']*buildings['ERA']
    buildings['DHW'] = buildings['k_DHW']*buildings['ERA']
    buildings['TOT_TH'] = buildings['SH'] + buildings['DHW']
    buildings['P_tot'] = buildings['TOT_TH'] / buildings["k_fehh"]

    return buildings

# TEST

In [214]:
UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

sup_file = DATA_DIRECTORY / "support_data" 
# reading consumption data for SH and DHW per type of building and construction time
data_sh = pd.read_csv(sup_file/"demand_SH.csv" , header=0, index_col=0)
data_dhw = pd.read_csv(sup_file/"demand_DHW.csv", header=0, index_col=0)
fehh = pd.read_csv(sup_file/"fehh.csv", header=0, index_col=0)

district_th = pd.DataFrame()

for dis_num in UST_district:
    communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]
    
    for com_num in communes:
        print(com_num)
        b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == com_num]# GGDENR Numéro OFS de la commune
        b = b.reset_index()
        b = b.drop(["GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1)

        b = post_processing(b)
        district_th = pd.concat([district_th,b],ignore_index=True)


    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    path = str(BLDG_DIRECTORY / CANTON)+"/"+ DISTRICT + ".csv"
    filepath = pathlib.Path(path)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    district_th.to_csv(filepath, sep=";", encoding='utf-8-sig', index_label='index')
    print("done with: ", DISTRICT)
    
    district_th = district_th[0:0]  

5001
5002
5003
5009
5010
5017
done with:  Bellinzona
5048
5049
5050
done with:  Blenio
5061
5063
5064
5071
5072
5073
5076
5077
5078
5079
done with:  Leventina
5091
5096
5097
5108
5112
5113
5115
5117
5118
5120
5121
5125
5131
5136
5138
5396
5397
5398
5399
done with:  Locarno
5141
5143
5144
5146
5148
5149
5151
5154
5160
5161
5162
5167
5171
5176
5180
5181
5186
5187
5189
5192
5193
5194
5196
5198
5199
5200
5203
5205
5206
5207
5208
5210
5212
5214
5216
5221
5225
5226
5227
5230
5231
5233
5236
5237
5238
5239
5240
done with:  Lugano
5242
5249
5250
5251
5254
5257
5260
5263
5266
5268
5269
done with:  Mendrisio
5281
5287
done with:  Riviera
5304
5307
5309
5310
5315
5317
5323
5324
done with:  Vallemaggia


## TEST in fragements

In [213]:
UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)


district_th = pd.DataFrame()

for dis_num in UST_district:
    communes = UST_comune.loc[UST_comune["Numero del Distretto"] == dis_num, "Numero UST del Comune"]
    
    for com_num in communes:
        #print(com_num)
        b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == com_num]# GGDENR Numéro OFS de la commune
        b = b.reset_index()
        b = b.drop(["GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1)
        district_th = pd.concat([district_th,b],ignore_index=True)


    DISTRICT = str(UST_comune.loc[(UST_comune["Numero del Distretto"] == dis_num), "Nome del Distretto"].iloc[0])
    path = str(BLDG_DIRECTORY / CANTON)+"/"+ DISTRICT + ".csv"
    print(path)
    filepath = pathlib.Path(path)  
    filepath.parent.mkdir(parents=True, exist_ok=True) 

Bellinzona Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Bellinzona.csv
Blenio Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Blenio.csv
Leventina Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Leventina.csv
Locarno Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Locarno.csv
Lugano Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Lugano.csv
Mendrisio Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Mendrisio.csv
Riviera Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Riviera.csv
Vallemaggia Bellinzona.csv C:\Users\jalomi.maayantardif\GitHub\ace-simulator\data\Buildings_rea\Ticino/Vallemaggia.csv


In [149]:
UST_file = "UST_comune.csv"
canton ="TI"
UST_comune = pd.read_csv(MAP_DIRECTORY/UST_file)
UST_comune = UST_comune[UST_comune['Cantone'] == canton].reset_index()
UST_district = UST_comune['Numero del Distretto'].drop_duplicates().reset_index(drop=True)

communes = UST_comune.loc[UST_comune["Numero del Distretto"] == UST_district[0], "Numero UST del Comune"]
communes

0    5001
1    5002
2    5003
3    5009
4    5010
5    5017
Name: Numero UST del Comune, dtype: int64

In [180]:
### WORKS ###

fileDir = DATA_DIRECTORY / "support_data" 
# reading consumption data for SH and DHW per type of building and construction time
data_sh = pd.read_csv(fileDir/"demand_SH.csv" , header=0, index_col=0)
data_dhw = pd.read_csv(fileDir/"demand_DHW.csv", header=0, index_col=0)
fehh = pd.read_csv(fileDir/"fehh.csv", header=0, index_col=0)
    
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.drop(["GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1)
b = b.reset_index()


age_group = b[["GKAT", "GKLAS", "GBAUP"]]
age_pivot = pd.pivot_table(age_group, values='GBAUP', index=['GKAT'], columns=['GKLAS'], aggfunc=np.median)
age_pivot = age_pivot.round(0)
col = list(age_pivot.columns)
for code in GKLAS:
    if code in col:
        pass
    else:
        age_pivot[code] = 8016.0

size_group = b[["GKAT", "GKLAS", "GAREA"]]
size_pivot = pd.pivot_table(size_group, values='GAREA', index=['GKAT'], columns=['GKLAS'], aggfunc=np.median)
size_pivot = size_pivot.round(0)


tech_group = b[["GKAT", "GKLAS", "GWAERZH1"]]
tech_pivot = pd.pivot_table(tech_group, values='GWAERZH1', index=['GKAT'], columns=['GKLAS'], aggfunc=np.median)
tech_pivot = tech_pivot.round(0)
col = list(tech_pivot.columns)
for code in GKLAS_HTG:
    if code in col:
        pass
    else:
        tech_pivot[code] = 7400.0

b["GKLAS_fill"] = b["GKLAS"]
b["GBAUP_fill"] = b["GBAUP"]
b["GAREA_fill"] = b["GAREA"]
b["GWAERZH1_fill"] = b["GWAERZH1"]

for g in range(len(b)):
    if (np.isnan(b.at[g,"GKLAS"])):
        if (b.at[g,"GKAT"] == 1020.0):
            b.loc[g,"GKLAS_fill"] = 1110.0
        elif (b.at[g,"GKAT"] == 1030.0):
            b.loc[g,"GKLAS_fill"] = 1122.0 # 63% of known 1040
        elif (b.at[g,"GKAT"] == 1040.0):
            b.loc[g,"GKLAS_fill"] = 1211.0  # 37% of known 1040
        elif (b.at[g,"GKAT"] == 1060.0):
            b.loc[g,"GKLAS_fill"] =(1271.0) # 31% of known 1060
                
                 
    if np.isnan(b.at[g,"GBAUP"]):         
        b.at[g,"GBAUP_fill"] = age_pivot.loc[(b.at[g,"GKAT"],b.at[g,"GKLAS_fill"])]

    if np.isnan(b.at[g,"GAREA"]):         
        b.at[g,"GAREA_fill"] = size_pivot.loc[(b.at[g,"GKAT"],b.at[g,"GKLAS_fill"])]
        
    if b.at[g,"GKLAS_fill"] in GKLAS_HTG:
        if np.isnan(b.at[g,"GWAERZH1"]):
            b.at[g,"GWAERZH1_fill"] = tech_pivot.loc[(b.at[g,"GKAT"],b.at[g,"GKLAS_fill"])]
    
#print(b.dtypes)
b["GAREA_fill"] = b["GAREA_fill"].astype(np.int64)
b["GKLAS_fill"] = b["GKLAS_fill"].astype(np.int64)
b["GBAUP_fill"] = b["GBAUP_fill"].astype(np.int64)

for g in range(len(b)):
    b.at[g,'k_SH'] = data_sh.loc[(b.at[g,"GBAUP_fill"],str(b.at[g,"GKLAS_fill"]))]
    b.at[g,'k_DHW'] = data_dhw.loc[(b.at[g,"GBAUP_fill"],str(b.at[g,"GKLAS_fill"]))]
    b.at[g,'k_fehh'] = fehh.loc[(b.at[g,"GBAUP_fill"],str(b.at[g,"GKLAS_fill"]))]

    
# calculating building attributes and demand
b['ERA'] = b['GAREA_fill']*b['GASTW']
b['SH'] = b['k_SH']*b['ERA']
b['DHW'] = b['k_DHW']*b['ERA']
b['TOT_TH'] = b['SH'] + b['DHW']
b['P_tot'] = b['TOT_TH'] / b['k_fehh']

b.head()

,index,EGID,GKODE,GKODN,GSTAT,GKAT,GKLAS,GBAUJ,GBAUP,GABBJ,...,GAREA_fill,GWAERZH1_fill,k_SH,k_DHW,k_fehh,ERA,SH,DHW,TOT_TH,P_tot
0,21905,11101274,2719448.610,1109711.580,1004.0,1060.0,1251.0,NaN,8014.0,NaN,...,500,7430.0,128.0,1.0,1700.0,1000.0,128000.0,1000.0,129000.0,75.882353
1,21906,11101275,2719553.730,1109908.500,1004.0,1020.0,1110.0,1993.0,8018.0,NaN,...,62,7430.0,60.0,14.0,2000.0,124.0,7440.0,1736.0,9176.0,4.588000
2,21907,11101276,2719578.443,1109805.775,1004.0,1030.0,1110.0,1946.0,8013.0,NaN,...,72,7410.0,141.0,14.0,2000.0,216.0,30456.0,3024.0,33480.0,16.740000
3,21908,11101278,2719408.349,1109673.037,1004.0,1060.0,1263.0,NaN,8012.0,NaN,...,311,7430.0,131.0,7.0,1700.0,933.0,122223.0,6531.0,128754.0,75.737647
4,21909,11101287,2720502.340,1110353.042,1004.0,1060.0,NaN,NaN,NaN,NaN,...,2431,NaN,0.0,0.0,8760.0,NaN,NaN,NaN,NaN,NaN


In [179]:
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.drop(["GWAERZW2","GENW2","GWAERDATW2", "GWAERDATH2", "GENH2", "GWAERZH2"], axis=1)
b = b.reset_index()


age_group = b[["GKAT", "GKLAS", "GBAUP"]]
age_pivot = pd.pivot_table(age_group, values='GBAUP', index=['GKAT'], columns=['GKLAS'], aggfunc=np.median)
age_pivot = age_pivot.round(0)
col = list(age_pivot.columns)
for code in GKLAS:
    if code in col:
        pass
    else:
        age_pivot[code] = 8016.0
age_pivot

GKLAS,1110.0,1121.0,1122.0,1211.0,1220.0,1242.0,1251.0,1252.0,1263.0,1271.0,...,1264.0,1265.0,1272.0,1241.0,1275.0,1276.0,1277.0,1278.0,1231.0,1273.0
GKAT,,,,,,,,,,,,,,,,,,,,,
1020.0,8013.0,8013.0,8013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0
1030.0,8013.0,8013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0
1040.0,NaN,NaN,NaN,8014.0,NaN,NaN,NaN,NaN,NaN,NaN,...,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0
1060.0,NaN,NaN,NaN,NaN,8015.0,8016.0,8016.0,8016.0,8012.0,8013.0,...,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0,8016.0


In [159]:
# FILING VALUES FOR THE MISSING TECHNOLOGY CELLS

b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

testhtg_group = b[["GKAT", "GKLAS", "GWAERZH1"]]
htg_pivot = pd.pivot_table(testhtg_group, values='GWAERZH1', index=['GKAT'], columns=['GKLAS'], aggfunc=np.mean)
htg_pivot = htg_pivot.round(0)
col = list(htg_pivot.columns)
for code in GKLAS_HTG:
    if code in col:
        pass
    else:
        htg_pivot[code] = 7400.0
htg_pivot = np.where(np.isnan(htg_pivot), 7400.0, htg_pivot)
htg_pivot

array([[7435., 7429., 7430., 7400., 7400., 7400., 7400., 7400., 7400.,
        7400., 7400., 7400., 7400., 7400., 7400., 7400., 7400.],
       [7410., 7430., 7400., 7400., 7400., 7400., 7400., 7400., 7400.,
        7400., 7400., 7400., 7400., 7400., 7400., 7400., 7400.],
       [7400., 7400., 7400., 7430., 7400., 7400., 7400., 7400., 7400.,
        7400., 7400., 7400., 7400., 7400., 7400., 7400., 7400.],
       [7400., 7400., 7400., 7400., 7430., 7430., 7430., 7403., 7400.,
        7400., 7400., 7400., 7400., 7400., 7400., 7400., 7400.]])

## GET SOME INFO/STATS

### ALL REA

In [17]:
GKLAS_count =  REA_TI_EXIST["GKLAS"].value_counts()
GKLAS_count 

GKLAS
1110.0    77503
1121.0    16981
1122.0    16814
1271.0    16346
1252.0    15624
1242.0    10119
1274.0     4436
1251.0     1899
1220.0     1249
1230.0     1045
1272.0      970
1211.0      607
1263.0      571
1241.0      207
1212.0      184
1265.0      184
1130.0      160
1261.0      123
1264.0       83
1278.0       79
1262.0       40
1231.0       34
1276.0        3
1277.0        2
1273.0        1
1275.0        1
Name: count, dtype: int64

In [108]:
TI_filter = REA_TI_EXIST
TI_filter = TI_filter[TI_filter["GKAT"]== 1030.0]
TI_filter = TI_filter["GKLAS"].value_counts()
TI_filter

GKLAS
1122.0    3412
1110.0    1009
1121.0     972
Name: count, dtype: int64

In [99]:
GKAT_count =  REA_TI_EXIST["GKAT"].value_counts()
GKAT_count 

GKAT
1020.0    105902
1060.0     61939
1030.0      5394
1040.0      2673
Name: count, dtype: int64

### PER COMUNE

In [161]:
#AGE PIVOT - MEAN

b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

testage_group = b[["GKAT", "GKLAS", "GBAUP"]]
age_pivot = pd.pivot_table(testage_group, values='GBAUP', index=['GKAT'], columns=['GKLAS'], aggfunc=np.mean)
age_pivot = age_pivot.round(0)
age_pivot

GKLAS,1110.0,1121.0,1122.0,1211.0,1220.0,1242.0,1251.0,1252.0,1263.0,1271.0,1274.0
GKAT,,,,,,,,,,,
1020.0,8014.0,8014.0,8013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1030.0,8013.0,8013.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040.0,NaN,NaN,NaN,8014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060.0,NaN,NaN,NaN,NaN,8015.0,8016.0,8016.0,8017.0,8012.0,8014.0,8015.0


In [168]:
# AGE GROUPBY - COUNT
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

GKAT_group = b[["GKAT", "GKLAS", "GBAUP"]]
GKAT_group = GKAT_group.groupby(["GKAT", "GKLAS"]).count()
GKAT_group

GBAUP
GKAT   GKLAS        
1020.0 1110.0    205
       1121.0     34
       1122.0      5
1030.0 1110.0      1
       1121.0      1
1040.0 1211.0      1
1060.0 1220.0      2
       1242.0     20
       1251.0      2
       1252.0     41
       1263.0      1
       1271.0     63
       1272.0      0
       1274.0      2

In [175]:
# AGE - GROUPBY, MEDIAN

b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

age_group = b[["GKLAS", "GBAUP"]]
#print(len(b))
GBAUP_group = age_group.groupby(["GKLAS"]).median()
GBAUP_group = pd.DataFrame(GBAUP_group).round(0)
GBAUP_group
GBAUP_group.index = GBAUP_group.index.astype(int)
GBAUP_group.index = GBAUP_group.index.astype(str)


testage_group = b[["GKAT", "GKLAS", "GBAUP"]]
tGBAUP_group = testage_group.groupby(["GKAT","GKLAS"]).median()
tGBAUP_group = pd.DataFrame(tGBAUP_group).round(0)
tGBAUP_group

GBAUP
GKAT   GKLAS         
1020.0 1110.0  8013.0
       1121.0  8013.0
       1122.0  8013.0
1030.0 1110.0  8013.0
       1121.0  8013.0
1040.0 1211.0  8014.0
1060.0 1220.0  8015.0
       1242.0  8016.0
       1251.0  8016.0
       1252.0  8016.0
       1263.0  8012.0
       1271.0  8013.0
       1272.0     NaN
       1274.0  8015.0

In [176]:
# AREA - GROUPBY, MEDIAN

b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

area_group = b[["GKLAS", "GAREA"]]
GAREA_group = area_group.groupby(["GKLAS"]).median()
GAREA_group = pd.DataFrame(GAREA_group).round(0)
GAREA_group
GAREA_group.index = GAREA_group.index.astype(int)
GAREA_group.index = GAREA_group.index.astype(str)
GAREA_group

,GAREA
GKLAS,
1110,65.0
1121,112.0
1122,92.0
1211,112.0
1220,117.0
1242,36.0
1251,341.0
1252,15.0
1263,311.0


In [170]:
# HTG - COUNT (GWAERZH1) - pivot with median value cannot be used in this case since not sequential (as opposed to AGE)
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()

htg_group = b[["GKAT", "GKLAS", "GWAERZH1"]]
htg_pivot = pd.pivot_table(htg_group, values='GWAERZH1', index=['GKAT'], columns=['GKLAS'], aggfunc="count")
htg_pivot = htg_pivot.round(0)
htg_pivot

GKLAS,1110.0,1121.0,1122.0,1211.0,1220.0,1242.0,1251.0,1252.0,1263.0,1271.0,1272.0,1274.0
GKAT,,,,,,,,,,,,
1020.0,198.0,34.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1030.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060.0,NaN,NaN,NaN,NaN,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0


### Check missing values (NaN)

In [171]:
## ALL REA
TI_filter = REA_TI_EXIST
TI_filter = TI_filter[TI_filter["GKAT"]== 1060.0]
TI_filter.isna().sum()

EGID              0
GKODE             5
GKODN             5
GSTAT             0
GKAT              0
GKLAS          9609
GBAUJ         33981
GBAUP          4892
GABBJ         61939
GAREA           114
GASTW          7462
GWAERZH1      56165
GENH1         56112
GWAERDATH1    56112
GWAERZH2      60786
GENH2         60786
GWAERDATH2    60786
GWAERZW1      57341
GENW1         57340
GWAERDATW1    57341
GWAERZW2      60397
GENW2         60397
GWAERDATW2    60397
GGDENR            0
GGDENAME          0
GDEKT             0
dtype: int64

In [172]:
## PER COMUNE
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()
b.isna().sum()

index           0
EGID            0
GKODE           0
GKODN           0
GSTAT           0
GKAT            0
GKLAS          63
GBAUJ         311
GBAUP          86
GABBJ         476
GAREA           0
GASTW          54
GWAERZH1      225
GENH1         225
GWAERDATH1    225
GWAERZH2      448
GENH2         448
GWAERDATH2    448
GWAERZW1      225
GENW1         225
GWAERDATW1    225
GWAERZW2      361
GENW2         361
GWAERDATW2    361
GGDENR          0
GGDENAME        0
GDEKT           0
dtype: int64

In [173]:
b = REA_TI_EXIST.loc[REA_TI_EXIST["GGDENR"] == 5009]# GGDENR Numéro OFS de la commune
b = b.reset_index()
b_res = b[b["GKAT"]== 1060.0]
b_res.isna().sum()

index           0
EGID            0
GKODE           0
GKODN           0
GSTAT           0
GKAT            0
GKLAS          60
GBAUJ         150
GBAUP          86
GABBJ         226
GAREA           0
GASTW          54
GWAERZH1      218
GENH1         218
GWAERDATH1    218
GWAERZH2      223
GENH2         223
GWAERDATH2    223
GWAERZW1      218
GENW1         218
GWAERDATW1    218
GWAERZW2      223
GENW2         223
GWAERDATW2    223
GGDENR          0
GGDENAME        0
GDEKT           0
dtype: int64